# Colored Water Pots Problem - First Project - Knowledge Representation

## Imports 

In [1]:
import os
import copy

## Classes 

### Node Class

**Node class constructor:**
* **info** - type: list(dictionary) - contains informations about the water pots 

    info =[ { "id" : number, "capacity" : number, "quantity" : number, "color" : string }, ... ]
    
* **cost** - type: number - is the cost of the path to the current node
* **heuristic**  - type: number - is the cost of the path from the current node to the destination node
* **estimated_cost** - type: number - is cost + heuristic
* **parent** : type: Node - is the current node's parent in the path
* **path_info** - type: dictionary - contains informations about the combanation of the pots

    path_info = { "first_pot" : potInfo, "second_pot" : potInfo, "color" : string, "poured_liters" : number }

In [2]:
class Node:
    
    # Class Constructor
    # Node (info, parent, cost, h, pathInfo)
    
    def __init__(self, info: list, parent,  cost = 0, heuristic = 0, path_info = {}):
        # type: list(dictionary) - contains informations about the water pots
        self.info = info
        
        # type: Node - is the current node's parent in the path
        self.parent = parent
        
        # type: number - is the cost of the path to the current node
        self.cost = cost
        
        # type: number - is the cost of the path from the current node to the destination node
        self.heuristic = heuristic
        
        # type: number - is cost + heuristic
        self.estimated_cost = self.cost + self.heuristic
        
        # type: type: dictionary- contains informations about the combanation of the pots
        self.path_info = path_info
    
    def __str__(self):
        result_string = ""
        
        # { "id" : number, "capacity" : number, "quantity" : number, "color" : string }
        for pot in self.info:
            values = list(pot.values())
            result_string += f"Pot -> id = {values[0]}; color = {values[3]}; quantity = {values[2]}; capacity = {values[1]}\n"
        
        return result_string
    
    def __repr__(self):
        result_string = f"Node info: {str(self.info)}"+ "\n"
        if self.parent != None:
            result_string += f"Parent: {str(self.parinte.info)}"+ "\n"
        else:
            result_string += f"Parent: None"+ "\n"
        result_string += f"Cost: {str(self.cost)}"+ "\n"
        result_string += f"Heuristic: {str(self.heuristic)}"+ "\n"
        result_string += f"Estimated Cost: {str(self.estimated_cost)}"+ "\n"
        result_string += f"Path Info: {str(self.path_info)}"+ "\n"
        return result_string                                   
    
    def get_path(self):
        node = self
        path = [node]
        
        while node.parent is not None:
            path.insert(0, node.parent)
            node = node.parent
        
        return path
    
    def print_path(self, print_cost = False, print_length = False):
        result_string = ""
        path = self.get_path()
        
        for node in path:
            path_info = node.path_info
            
            if path_info != None and node.parent is not None:
                values = list(path_info.values())
                # { "first_pot" : potInfo, "second_pot" : potInfo, "color" : string, "poured_liters" : number }
                result_string += f"{values[3]} liters of colored water {values[2]} were poured from the pot {values[0]} into the pot {values[1]}\n"
                result_string = node.__str__() + "\n"
                
        if print_cost:
            result_string += "Path cost: " + str(self.estimated_cost) + "\n"
        if print_length:
            result_string += "Path length: " + str(len(path)) + "\n"
            
        return result_string
    
    def contain_path(self, checked_node):
        node = self

        while node is not None:
            if node.info == checked_node.info:
                return True
            node = node.parent
        
        return False
    
    def has_final_state(self, final_state):
        for state_pot in final_state:
            # {"quantity" :  number, "color" : string}
            values = list(state_pot.values())
            ok = False
            for pot in self.info:
                # { "id" : number, "capacity" : number, "quantity" : number, "color" : string }
                pot_values = list(pot.values())
                if values[0] == pot_values[2] and values[1] == pot_values[3]:
                    ok = True
                    break
            if not ok:
                return ok
        return True
                
    

In [3]:
# For testing
node = Node([{ "id" : 1, "capacity" : 10, "quantity" : 8, "color" : "red" }], None)
print(node.__str__())
print(node.__repr__())
print(node.has_final_state([{"quantity" :  8, "color" : "red"}]))
print(node.has_final_state([{"quantity" :  8, "color" : "blue"}]))

Pot -> id = 1; color = red; quantity = 8; capacity = 10

Node info: [{'id': 1, 'capacity': 10, 'quantity': 8, 'color': 'red'}]
Parent: None
Cost: 0
Heuristic: 0
Estimated Cost: 0
Path Info: {}

True
False


In [4]:
class Graph:
    def __init__(self, input_filename, output_filename, output_path, timeout):
        self.timeout = timeout
        try:
            reader = open(input_filename, "r")
            
            try:
                file_content = reader.read()
                
                initial_info, start_state, final_state = split_file(file_content)
                
                self.color_combiantions, self.color_cost = self.parse_initial_info(initial_info)
                
                self.start_state = self.parse_start_state(start_state)
                
                self.final_state = self.parse_final_state(final_state)
            except:
                print(f"Could not parse the file! Filename:{input_filename}")
                sys.exit(0)
        except:
            print(f"Could not open the input file! Filename:{input_filename}")
            sys.exit(0)
            
    def split_file(file_content):
        first_split = file_content.strip().split("start_state")
        second_split = first_split.strip().split("final_state")
        
        initial_info = first_split[0].strip().split('\n')
        start_state = second_split[0].strip().split('\n')
        final_state = second_split[1].strip().split('\n')
        
        return initial_info, start_state, final_state
    
    def parse_initial_info(initial_info):
        color_combinations = list()
        color_cost = dict()
        
        for line in initial_info:
            content = line.split()
            if len(content) == 2:
                color_cost[content[0]] = int(content[1])
            else:
                color_combinations.append(tuple(content[0], content[1], content[2]))
        return color_combinations, color_cost
    
    def parse_start_state(start_state):
        state = []
        count = 0
        for line in start_state:
            content = line.split()
            if len(content) == 2:
                state.append(tuple(count, int(content[0]), 0, None))
            else:
                if content[2] not in color_cost.keys():
                    raise Exception
                state.append(tuple(count, int(content[0]), int(content[1], content[2])))
            count += 1
        return state
    
    def parse_final_state(final_state):
        state = []
        for line in final_state:
            content = line.split()
            if content[1] not in color_cost.keys():
                    raise Exception
            state.append(tuple(int(content[0]),content[1]))
        return state
    
    def find_distinct_colors(self):
        found_colors = set()
        for (color1, color2, color3) in self.color_combinations:
            found_colors.append(color1)
            found_colors.append(color2)        
        return found_colors
    
    def count_combinations(self, info):
        found_colors = self.find_distinct_colors()
        number = 0
        for color in found_colors:
            for pot in info:
                values = list(pot.values())
                # "Pot -> id = {values[0]}; color = {values[3]}; quantity = {values[2]}; capacity = {values[1]}\n"
                if values[3] == color:
                    number += 1
                    break
        return number
            
    def count_final(self, info):
        number = 0
        for (color1, color2, color3) in self.color_combinations:
            for pot in info:
                values = list(pot.values())
                # "Pot -> id = {values[0]}; color = {values[3]}; quantity = {values[2]}; capacity = {values[1]}\n"
                if values[3] == color3:
                    number += 1
                    break
        return number
    
    def check_final(self, info):
        for (quantity, color) in self.final_state:
            count_errors = 0
            for pot in info:
                values = list(pot.values())
                if quantity > values[2]:
                    count_errors += 1
                if count_errors == len(info):
                    return False
        return True
    
    def check_node(self, info):
        count_final_colors = self.count_final(info)
        count_color_combinations = self.count_combinations(info)
        check_final_state = self.check_final(info)
        
        if count_final_colors == len(self.color_combinations) and count_color_combinations == 0:
            return True
        elif count_final_colors == 0 and count_color_combinations >= 2:
            return True
        elif count_color_combinations != 0 and count_final_colors != 0:
            return True
        
        return False
    
    
        